# 날짜 범위 내의 제주도 강수량 크롤링
- https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=189&yy=2019&mm=9&obs=1

In [43]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [16]:
def crawling(url):
    page = requests.get(url)

    soup = BeautifulSoup(page.text, "html.parser")
    # contents = soup.find_all('table','table-col table-cal')
    contents = soup.find_all('span')

    return contents

In [17]:
def preprocessing(contents,result):
    for i in contents:
        if '일강수량:' in i.text:
            value = i.text.split(':')[1]
            if value == ' - ' or value == '0.0mm':
                value = 0
                result.append(value)
                continue
            result.append(float(value[:-2]))
    return result

In [18]:
def convert_url(loc,year,month):
    if loc == 'jeju':
        loc = 184
    elif loc == 'seogwipo':
        loc = 189

    url = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn={}&yy={}&mm={}&obs=1".format(loc,year,month)
    return url    


In [62]:
# url_jeju = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=184&yy=2019&mm=9&obs=1"
# url_seogwipo = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=189&yy=2019&mm=9&obs=1"

In [19]:
url_jeju_9 = convert_url('jeju',2019,9)
url_jeju_10 = convert_url('jeju',2019,10)

url_seogwipo_9 = convert_url('seogwipo',2019,9)
url_seogwipo_10 = convert_url('seogwipo',2019,10)

In [24]:
jeju_9, seogwipo_9 = [],[]

contents = crawling(url_jeju_9)
rain_jeju_9 = preprocessing(contents,jeju_9)

contents = crawling(url_seogwipo_9)
rain_seogwipo_9 = preprocessing(contents,seogwipo_9)

In [25]:
jeju_10, seogwipo_10 = [],[]

contents = crawling(url_jeju_10)
rain_jeju_10 = preprocessing(contents,jeju_10)

contents = crawling(url_seogwipo_10)
rain_seogwipo_10 = preprocessing(contents,seogwipo_10)

In [37]:
ts_ms = pd.date_range(start = '2019-09-01',     # 날짜 범위 시작
                     end = '2019-10-31',                # 날짜 범위 끝
                    #  periods = 45,               # 생성할 Timestamp 개수
                     freq = 'D',               # 시간 간격(MS : 월의 시작일)
                     tz = 'Asia/Seoul')         # 시간대(timezone)

In [38]:
len(ts_ms)

61

In [47]:
data = pd.DataFrame(columns = ['date','rain_jeju(mm)','rain_seogwipo(mm)'])

data['date'] = ts_ms

In [48]:
data.head()

,date,rain_jeju(mm),rain_seogwipo(mm)
0,2019-09-01 00:00:00+09:00,NaN,NaN
1,2019-09-02 00:00:00+09:00,NaN,NaN
2,2019-09-03 00:00:00+09:00,NaN,NaN
3,2019-09-04 00:00:00+09:00,NaN,NaN
4,2019-09-05 00:00:00+09:00,NaN,NaN


In [49]:
print(len(rain_jeju_9 + rain_jeju_10))
print(len(rain_seogwipo_9 + rain_seogwipo_10))

61
61


In [50]:
data['rain_jeju(mm)'] = rain_jeju_9 + rain_jeju_10
data['rain_seogwipo(mm)'] = rain_seogwipo_9 + rain_seogwipo_10

In [51]:
data.head()

,date,rain_jeju(mm),rain_seogwipo(mm)
0,2019-09-01 00:00:00+09:00,26.1,18.7
1,2019-09-02 00:00:00+09:00,25.7,14.6
2,2019-09-03 00:00:00+09:00,13.9,9.5
3,2019-09-04 00:00:00+09:00,43.0,28.9
4,2019-09-05 00:00:00+09:00,7.0,15.7


In [52]:
data.to_csv("rain_jeju_9&10.csv",index = False)